In [ ]:
import torch
from diffusers import StableDiffusion3Pipeline

token = "hf_###..."  # 토큰 입력
model_name = "stabilityai/stable-diffusion-3-medium-diffusers"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
pipe = StableDiffusion3Pipeline.from_pretrained(
    model_name, token=token, torch_dtype=torch.float16
).to(device)
print(pipe.config)

In [ ]:
prompt = ["gray cat holding a yellow ball"]
prompt_2 = prompt
prompt_3 = prompt
num_images_per_prompt = 1
clip_skip = None
max_sequence_length = 256

prompt_embed, pooled_prompt_embed = pipe._get_clip_prompt_embeds(
    prompt=prompt,
    device="cuda",
    num_images_per_prompt=num_images_per_prompt,
    clip_skip=clip_skip,
    clip_model_index=0
)
prompt_2_embed, pooled_prompt_2_embed = pipe._get_clip_prompt_embeds(
    prompt=prompt_2,
    device="cuda",
    num_images_per_prompt=num_images_per_prompt,
    clip_skip=clip_skip,
    clip_model_index=1
)
clip_prompt_embeds = torch.cat([prompt_embed, prompt_2_embed], dim=-1)

t5_prompt_embed = pipe._get_t5_prompt_embeds(
    prompt=prompt_3,
    num_images_per_prompt=num_images_per_prompt,
    max_sequence_length=max_sequence_length,
    device="cuda"
)

clip_prompt_embeds = torch.nn.functional.pad(
    clip_prompt_embeds, (0, t5_prompt_embed.shape[-1] - clip_prompt_embeds.shape[-1])
)

prompt_embeds = torch.cat([clip_prompt_embeds, t5_prompt_embed], dim=-2)
pooled_prompt_embeds = torch.cat([pooled_prompt_embed, pooled_prompt_2_embed], dim=-1)

print("prompt_embed.shape:", prompt_embed.shape)
print("prompt2_embed.shape:", prompt_2_embed.shape)
print("t5_prompt_embed.shape:", t5_prompt_embed.shape)
print("prompt_embeds.shape:", prompt_embeds.shape)
print("pooled_prompt_embeds.shape:", pooled_prompt_embeds.shape)

In [ ]:
for main_name, main_module in pipe.transformer.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│    └", sssub_name)